In [1]:
import sys
sys.path.append('../')
from rl_recruiter.rl_model import RL_Recruiter_plus
from rl_recruiter.entropy_cal import type_2_entro

In [5]:
type_2_entro('./data/trajectory.json', 'predictability.json')

In [2]:
rlp = RL_Recruiter_plus('./data/para_settings.json', './data/thres.json')

In [6]:
rlp.train_and_evaluate('./data/trajectory.json', 'predictability.json')

Loading trajectory Data...
Getting eligible user list
init record set
load entro_daily
train and evaluate from day to day


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [03:12<00:00, 13.75s/it]

Each day's coverage:
[664, 778, 1090, 1191, 1267, 983, 1256, 1339, 1501, 1259, 1234, 1381, 1035, 1072]
